In [31]:
import os
os.chdir('D:\Deep Learning\Tensorflow & Keras\deploy2\project3')

In [32]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [33]:
from TomatoDiseases.constant import *
from TomatoDiseases.utils.common import read_yaml, create_directories

In [34]:
class ConfigManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir= config.root_dir,
            base_model_path= config.base_model_path,
            updated_base_model_path= config.updated_base_model_path,
            params_image_size= self.params.IMAGE_SIZE,
            params_learning_rate= self.params.LEARNING_RATE,
            params_include_top= self.params.INCLUDE_TOP,
            params_weights= self.params.WEIGHTS,
            params_classes= self.params.CLASSES
        )
        
        return prepare_base_model_config

In [35]:
import os
import tensorflow as tf
import urllib.request as request
from TomatoDiseases import logger
from TomatoDiseases.utils.common import *

In [36]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            include_top= self.config.params_include_top,
            weights= self.config.params_weights,
            input_shape= self.config.params_image_size
        )
        
        self.save_model(path= self.config.base_model_path, model= self.model)

    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        
        elif (freeze_till is not None) or (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
                
        flatten_in = tf.keras.layers.Flatten()(model.output)
        flcn_layers = tf.keras.layers.Dense(units=classes, activation='softmax')(flatten_in)
        
        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = flcn_layers
        )
        
        full_model.compile(
            optimizer= tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ['recall']
        )
        
        full_model.summary()
        
        return full_model

    def updated_base_model(self):
        self.full_model = self.prepare_full_model(
            model= self.model,
            classes= self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate= self.config.params_learning_rate
        )
        
        self.save_model(path= self.config.updated_base_model_path, model= self.full_model)
    
    @staticmethod
    def save_model(path: Path,
                   model: tf.keras.Model):
        model.save(path)
        

In [37]:
try:
    config = ConfigManager()
    prepare_base_model_config = config.prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.updated_base_model()
except Exception as e:
    raise e

[2023-07-10 23:08:33,313: INFO: common: yaml file: config.yaml loaded successfully]
[2023-07-10 23:08:33,316: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-10 23:08:33,317: INFO: common: created directory at: artifacts]
[2023-07-10 23:08:33,318: INFO: common: created directory at: artifacts/prepare_base_model]
[2023-07-10 23:08:33,680: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)    

c:\Users\Admin\anaconda3\envs\tomato\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
